##To do:
1. Dealing with nulls on foreign keys(if applicable) (Done)
2. Joining variables + Mark which variables are continuous/categorical (rename to start with cat)(in progress)
3. Adjust columns:
+ Mine data from Strength_Code (Done)
+ adjust Script/Dispense_Qty (Done)
+ multiply unit with ddd for uniform representation (Done)

5. Create additional features
+ Year (cont) (Done)
+ Month (cat) (Done)
+ Season (cat)
+ Date(Year+Month) (cont) (Done)

4. Check which variables to leave:
+ Contigious data - nulls < 50%
+ from postcodes: state, lat, lon, (regional_cat as both cont and cat) (Done)
+ Not too many subgroups < 20 - script running through a variable:
    + Check amount of columns
    + For each column select distinct
    + If <20 check distribution between groups - bar plot
    + If continuous data - create boxplot
+ Not repeated throughout the set
6. Analyse correlations between variables:
+ covariance matrix (heatmap) - for continuous
+ pearson chi square between categorical/ cramer's v

1. Foreign keys:
+ Drug_ID -> drug_lookup.MasterProductID -> chronic_illness_lookup.MasterProductID
+ drug_lookup.ATCLevel5Code -> defined_daily_doses.ATC_CODE
+ Patient_ID -> patient_lookup.Patient_ID
+ Store_ID -> store_lookup.Store_ID
+ patient_lookup.postcode -> postcodes.postcode
+ stores_lookup.postcode -> postcodes.postcode

In [249]:
import numpy as np
import pandas as pd
import re
from ggplot import *
from collections import Counter

project_id = "the-d-team-164512"

In [250]:
foreign_tab = ['transactions_all_c2','transactions_all_c2','transactions_all_c2',' drug_lookup', 'drug_lookup',
               'chronic_illness_lookup', 'defined_daily_doses','patients_lookup2', 'stores_lookup2', 'postcodes2']
foreign_var = ['Drug_ID ','Patient_ID','Store_ID','MasterProductID', 'ATCLevel5Code', 'MasterProductID',
              'ATC_CODE','Patient_ID', 'Store_ID', 'postcode']
print(len(foreign_var))
len(foreign_tab)

10


10

In [59]:
results = list()

for i in np.arange(len(foreign_tab)):
    query = "SELECT COUNT(1) as cnt from dthon." + foreign_tab[i] + " WHERE " + foreign_var[i] + \
    " IS NULL OR LOWER(CAST(" + foreign_var[i] + """ as STRING)) = "na" GROUP BY """ + foreign_var[i]
    results.append(pd.read_gbq(query, project_id,dialect='standard'))
    

Requesting query... ok.
Query running...
Query done.
Processed: 473.2 Mb

Retrieving results...
Got 0 rows.

Total time taken 3.78 s.
Finished at 2017-04-24 17:55:14.
Requesting query... ok.
Query running...
Query done.
Processed: 473.2 Mb

Retrieving results...
Got 0 rows.

Total time taken 3.86 s.
Finished at 2017-04-24 17:55:19.
Requesting query... ok.
Query running...
Query done.
Processed: 473.2 Mb

Retrieving results...
Got 0 rows.

Total time taken 3.12 s.
Finished at 2017-04-24 17:55:24.
Requesting query... ok.
Query running...
Query done.
Processed: 107.8 kb

Retrieving results...
Got 0 rows.

Total time taken 3.07 s.
Finished at 2017-04-24 17:55:29.
Requesting query... ok.
Query running...
Query done.
Processed: 108.1 kb

Retrieving results...
Got 1 rows.

Total time taken 3.48 s.
Finished at 2017-04-24 17:55:35.
Requesting query... ok.
Query running...
Query done.
Processed: 17.2 kb

Retrieving results...
Got 0 rows.

Total time taken 3.64 s.
Finished at 2017-04-24 17:55:41.

In [253]:
a = [num for num in np.arange(len(results)) if len(results[num])!=0]
for num in a:
    print(foreign_tab[num] + "." + foreign_var[num] + ": " + str(results[num]['cnt'][0]))

drug_lookup.ATCLevel5Code: 2
postcodes2.postcode: 2


In [252]:
foreign_tab = [' drug_lookup', 'drug_lookup','chronic_illness_lookup', 'defined_daily_doses','patients_lookup2',
               'stores_lookup2', 'postcodes2', 'postcode_viz']
foreign_var = ['MasterProductID', 'ATCLevel5Code', 'MasterProductID', 'ATC_CODE','Patient_ID', 
               'Store_ID', 'postcode', 'postcode']

results = list()

for i in np.arange(len(foreign_tab)):
    query = "SELECT " + foreign_var[i] + ", COUNT(1) as cnt from dthon." + foreign_tab[i] + \
    " GROUP BY " + foreign_var[i] + " HAVING cnt > 1"
    results.append(pd.read_gbq(query, project_id,dialect='standard'))

Requesting query... ok.
Query running...
Query done.
Processed: 107.8 kb

Retrieving results...
Got 0 rows.

Total time taken 2.8 s.
Finished at 2017-04-25 14:14:31.
Requesting query... ok.
Query running...
Query done.
Processed: 108.1 kb

Retrieving results...
Got 976 rows.

Total time taken 4.66 s.
Finished at 2017-04-25 14:14:38.
Requesting query... ok.
Query running...
Query done.
Processed: 17.2 kb

Retrieving results...
Got 0 rows.

Total time taken 2.86 s.
Finished at 2017-04-25 14:14:43.
Requesting query... ok.
Query running...
Query done.
Processed: 42.4 kb

Retrieving results...
Got 0 rows.

Total time taken 3.87 s.
Finished at 2017-04-25 14:14:49.
Requesting query... ok.
Query running...
Query done.
Processed: 4.3 Mb

Retrieving results...
Got 0 rows.

Total time taken 2.8 s.
Finished at 2017-04-25 14:14:53.
Requesting query... ok.
Query running...
Query done.
Processed: 22.0 kb

Retrieving results...
Got 0 rows.

Total time taken 3.34 s.
Finished at 2017-04-25 14:14:58.
Req

In [255]:
results[6]

,postcode,cnt
0,4726,2
1,3113,2
2,7175,2
3,5155,2
4,6071,2
5,3054,2
6,2110,2
7,2133,2
8,4743,2
9,3070,2


Ok, the amount of nulls/NAs is not high and the most crucial keys are fully covered.
Let's perform the joins.

Now, let's augment the data by extracting the values from Strength Codes.

In [167]:
query = """
select StrengthCode as scd, count(1)
from dthon.drug_lookup dl
group by StrengthCode
"""

codes = pd.read_gbq(query, project_id, dialect="standard")

Requesting query... ok.
Query running...
Query done.
Processed: 118.4 kb

Retrieving results...
Got 2804 rows.

Total time taken 5.3 s.
Finished at 2017-04-24 21:49:19.


In [214]:
p = re.compile('(\A[0-9.()]*[mc]*[gl])(?:[^mgl]*)(?![mgl])$', re.IGNORECASE)
dosage = [p.search(str(obj)) for obj in codes['scd']]
dosage_cmpl = [obj.groups()[0] if obj is not None else 'NA' for obj in dosage]
dosage_cmpl[1633] = 'NA'
p = re.compile('[0-9.]*')
dosage_val = [float(p.search(obj)[0]) if obj!='NA' else 0 for obj in dosage_cmpl]
p = re.compile('[mcgl]+', re.IGNORECASE)
dosage_mg = [p.search(obj)[0] if obj!='NA' else 'NA' for obj in dosage_cmpl]

In [215]:
Counter(dosage_mg)

Counter({'G': 130, 'L': 4, 'MCG': 116, 'MG': 235, 'ML': 78, 'NA': 2241})

In [216]:
def conversion(obj):
    if obj.lower() in ['mg', 'ml']:
        return 1
    elif obj.lower() in ['mcg', 'mcl']:
        return 0.001
    elif obj.lower() in ['g', 'l']:
        return 1000
    else:
        return 0
dosage_mg2 = [conversion(obj) for obj in dosage_mg]

In [217]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
tester = [obj for obj in dosage_val if not is_number(obj)]
tester

[]

In [199]:
print([num for num in range(len(dosage_val)) if dosage_val[num] == ''])

[1633]

In [204]:
codes['scd'][1633]

'L 100 4209'

In [247]:
final = [x * y for x,y in zip(dosage_val, dosage_mg2)]
final = [obj if obj != 0 else None for obj in final]
df_codes = pd.DataFrame(data=codes['scd'], columns = ["Code"])
df_dose = pd.DataFrame(data=final, columns = ["Calculated_Dose"])
df_final = pd.concat([codes['scd'], df_dose], axis=1)
df_final[0:10]

,scd,Calculated_Dose
0,10CM X 4M 8635,NaN
1,"5ML, 4",5.0
2,250MG 2ML AOVEK,NaN
3,S012023 4,NaN
4,10CM X 12CM 10 4629,NaN
5,250MG/1.25MG,NaN
6,14CM SPIRAL 10 H7983,NaN
7,"10.8MG/50MG, 28",NaN
8,67MG,67.0
9,2% (20MG/ML) 30ML,NaN


In [248]:
df_final.to_gbq('dthon.dose_calc', project_id)




Streaming Insert is 100.0% Complete


